# TFX Adult Pipeline
Kernel: **Python (tfx-wsl-py39)**


In [15]:
import os, pathlib

HOME = str(pathlib.Path.home())

PROJECT_ROOT      = "/mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline"
PIPELINE_ROOT     = os.path.join(PROJECT_ROOT, "yrbror-pipeline")
SERVING_MODEL_DIR = os.path.join(PROJECT_ROOT, "serving_model_dir")

DATA_PATH = os.path.join(PROJECT_ROOT, "data", "adult.csv")

TRANSFORM_MODULE  = os.path.join(PROJECT_ROOT, "pipeline", "preprocessing_clean.py")
TRAINER_MODULE    = os.path.join(PROJECT_ROOT, "pipeline", "model.py")

print("PROJECT_ROOT      :", PROJECT_ROOT)
print("DATA_PATH         :", DATA_PATH)
print("DATA_PATH exists? :", os.path.exists(DATA_PATH))
print("TRANSFORM_MODULE  :", TRANSFORM_MODULE, os.path.exists(TRANSFORM_MODULE))
print("TRAINER_MODULE    :", TRAINER_MODULE,  os.path.exists(TRAINER_MODULE))
print("PIPELINE_ROOT     :", PIPELINE_ROOT)
print("SERVING_MODEL_DIR :", SERVING_MODEL_DIR)

PROJECT_ROOT      : /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline
DATA_PATH         : /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/data/adult.csv
DATA_PATH exists? : True
TRANSFORM_MODULE  : /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/pipeline/preprocessing_clean.py True
TRAINER_MODULE    : /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/pipeline/model.py True
PIPELINE_ROOT     : /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/yrbror-pipeline
SERVING_MODEL_DIR : /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/serving_model_dir


In [16]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration import metadata
import os

# Metadata disimpan di dalam folder proyek biar rapi
METADATA_PATH = os.path.join(PROJECT_ROOT, "metadata", "metadata.db")

context = InteractiveContext(
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=metadata.sqlite_metadata_connection_config(
        METADATA_PATH
    ),
)

context

## 1) ExampleGen

In [17]:
from tfx.proto import example_gen_pb2
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
import os

input_config = example_gen_pb2.Input(
    splits=[example_gen_pb2.Input.Split(name="train", pattern="adult.csv")]
)

example_gen = CsvExampleGen(
    input_base=os.path.dirname(DATA_PATH),
    input_config=input_config,
)

ex_result = context.run(example_gen)
ex_result

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 56
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Pada tahap ini, ExampleGen digunakan untuk membaca dataset CSV dan mengubahnya ke format TFRecord. Format TFRecord lebih efisien untuk diproses dalam pipeline TFX. ExampleGen juga melakukan pembagian dataset (train/eval) secara otomatis.

## 2) StatisticsGen → SchemaGen → ExampleValidator

In [18]:
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.example_validator.component import ExampleValidator

statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
schema_gen     = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                           infer_feature_shape=True)
example_val    = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'],
)

context.run(statistics_gen)
context.run(schema_gen)
context.run(example_val)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 59
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

StatisticsGen digunakan untuk menghitung statistik dari fitur pada dataset, seperti mean, min-max, frekuensi kategori, dan lainnya. Statistik ini digunakan pada tahap SchemaGen dan ExampleValidator.

SchemaGen menghasilkan schema otomatis berdasarkan statistik dataset. Schema akan berisi tipe data setiap fitur, domain nilai, serta informasi apakah sebuah kolom dapat memiliki missing value atau tidak.

ExampleValidator melakukan validasi dataset terhadap schema. Tahap ini membantu mendeteksi anomali, data corrupt, nilai outlier, atau data yang hilang dalam jumlah besar.

## 3) Transform

In [19]:
from tfx.components.transform.component import Transform
from tfx.types import artifact_utils

transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=TRANSFORM_MODULE,
)

tr_result = context.run(transform)
tr_result

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying model.py -> build/lib
copying preprocessing.py -> build/lib
copying preprocessing_clean.py -> build/lib
installing to /tmp/tmp702uu_6v
running install
running install_lib


/mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/tfx-venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


copying build/lib/preprocessing_clean.py -> /tmp/tmp702uu_6v
copying build/lib/model.py -> /tmp/tmp702uu_6v
copying build/lib/preprocessing.py -> /tmp/tmp702uu_6v
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp702uu_6v/tfx_user_code_Transform-0.0+f28aa178105ced362a92d3f4fd7fe13bc25aae5df8fd76d67fbdb708e89690f3.egg-info
running install_scripts
creating /tmp/tmp702uu_6v/tfx_user_code_Transform-0.0+f28aa178105ced362a92d3f4fd7fe13bc25aae5df8fd76d67fbdb708e89690f3.dist-info/WHEEL
creating '/tmp/tmp

2025-11-15 22:46:56.260979: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: UNKNOWN ERROR (100)


INFO:tensorflow:Assets written to: /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/yrbror-pipeline/Transform/transform_graph/60/.temp_path/tftransform_tmp/45b5195b82e948ff860f8279830f6e41/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 60
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Transform menggunakan TensorFlow Transform (tft) untuk melakukan preprocessing pada data. Proses mencakup:
- Normalisasi data numerik (z-score)
- Membentuk vocabulary untuk fitur kategori
- Mengubah data kategori menjadi indeks
- Melakukan imputasi missing value

Transform menghasilkan dua output:
1. Transform graph untuk dipakai Trainer
2. Transformed examples untuk data hasil preprocessing

In [13]:
tr_graph_artifact = transform.outputs["transform_graph"].get()[0]
tr_examples_artifact = transform.outputs["transformed_examples"].get()[0]

tr_graph_uri = tr_graph_artifact.uri
tr_examples_uri = tr_examples_artifact.uri

print("transform_graph       ->", tr_graph_uri)
print("transformed_examples  ->", tr_examples_uri)

transform_graph       -> /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/artifacts/Transform/transform_graph/51
transformed_examples  -> /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/artifacts/Transform/transformed_examples/51


## 4) Trainer

In [14]:
import pandas as pd

df = pd.read_csv(DATA_PATH)
df.columns.tolist()

['age',
 'workclass',
 'fnlwgt',
 'education',
 'educational-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'income']

In [15]:
from tfx.components.trainer.component import Trainer
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=TRAINER_MODULE,
    transformed_examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    train_args=trainer_pb2.TrainArgs(num_steps=2000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500),
)

trn_result = context.run(trainer)
trn_result

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying model.py -> build/lib
copying preprocessing.py -> build/lib
copying preprocessing_clean.py -> build/lib
installing to /tmp/tmp_cxz4tg_
running install
running install_lib
copying build/lib/preprocessing_clean.py -> /tmp/tmp_cxz4tg_
copying build/lib/model.py -> /tmp/tmp_cxz4tg_
copying build/lib/preprocessing.py -> /tmp/tmp_cxz4tg_
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmp_cxz4tg_/tfx_user_code_Trainer-0.0+f28aa

/mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/tfx-venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


creating /tmp/tmp_cxz4tg_/tfx_user_code_Trainer-0.0+f28aa178105ced362a92d3f4fd7fe13bc25aae5df8fd76d67fbdb708e89690f3.dist-info/WHEEL
creating '/tmp/tmpvtljy8ux/tfx_user_code_Trainer-0.0+f28aa178105ced362a92d3f4fd7fe13bc25aae5df8fd76d67fbdb708e89690f3-py3-none-any.whl' and adding '/tmp/tmp_cxz4tg_' to it
adding 'model.py'
adding 'preprocessing.py'
adding 'preprocessing_clean.py'
adding 'tfx_user_code_Trainer-0.0+f28aa178105ced362a92d3f4fd7fe13bc25aae5df8fd76d67fbdb708e89690f3.dist-info/METADATA'
adding 'tfx_user_code_Trainer-0.0+f28aa178105ced362a92d3f4fd7fe13bc25aae5df8fd76d67fbdb708e89690f3.dist-info/WHEEL'
adding 'tfx_user_code_Trainer-0.0+f28aa178105ced362a92d3f4fd7fe13bc25aae5df8fd76d67fbdb708e89690f3.dist-info/top_level.txt'
adding 'tfx_user_code_Trainer-0.0+f28aa178105ced362a92d3f4fd7fe13bc25aae5df8fd76d67fbdb708e89690f3.dist-info/RECORD'
removing /tmp/tmp_cxz4tg_


Processing ./yrbror-pipeline/artifacts/_wheels/tfx_user_code_Trainer-0.0+f28aa178105ced362a92d3f4fd7fe13bc25aae5df8fd76d67fbdb708e89690f3-py3-none-any.whl
=== TRAIN FILES PATTERN ===
['/mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/artifacts/Transform/transformed_examples/51/Split-train/*']
Sample train files: ['/mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/artifacts/Transform/transformed_examples/51/Split-train/transformed_examples-00000-of-00001.gz']
=== EVAL FILES PATTERN ===
['/mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/artifacts/Transform/transformed_examples/51/Split-eval/*']
Sample eval files : ['/mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/artifacts/Transform/transformed_examples/51/Split-eval/transformed_examples-00000-of-00001.gz']
Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


2000/2000 [==============================] - 14s 6ms/step - loss: 0.3191 - accuracy: 0.8527 - val_loss: 0.2987 - val_accuracy: 0.8618


INFO:tensorflow:Assets written to: /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/artifacts/Trainer/model/52/Format-Serving/assets


INFO:tensorflow:Assets written to: /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/artifacts/Trainer/model/52/Format-Serving/assets


Model saved to: /mnt/c/Users/laptop anyar/Downloads/tfx_adult_pipeline/yrbror-pipeline/artifacts/Trainer/model/52/Format-Serving


ExecutionResult(
    component_id: Trainer
    execution_id: 52
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Pada tahap ini, model deep learning dibangun menggunakan Keras Sequential. Trainer menggunakan:
- Data yang sudah ditransformasi
- Transform graph dari tahap Transform
- Arsitektur model:
    Dense(128) : ReLU
    Dense(64) : ReLU
    Dense(32) : ReLU
    Output → Sigmoid
Model dilatih menggunakan binary crossentropy dan dievaluasi menggunakan metrik akurasi.

## 5) Evaluator

In [22]:
from tfx.components import Evaluator
from tfx.proto import evaluator_pb2
import tensorflow_model_analysis as tfma

ACCURACY_THRESHOLD = 0.75    

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            label_key="income"   
        )
    ],
    slicing_specs=[
        tfma.SlicingSpec()     
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(
                    class_name="BinaryAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": ACCURACY_THRESHOLD}
                        )
                    )
                )
            ]
        )
    ],
)

evaluator = Evaluator(
    examples=transform.outputs["transformed_examples"],
    model=trainer.outputs["model"],
    eval_config=eval_config,
)

eval_result = context.run(evaluator)
eval_result

ExecutionResult(
    component_id: Evaluator
    execution_id: 55
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Evaluator menggunakan TensorFlow Model Analysis (TFMA) untuk mengevaluasi performa model. Evaluator menghasilkan laporan metrik akurasi, slicing metrics, dan memberikan status 'blessed' jika model memenuhi syarat.

## 6) Resolver

In [11]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import (
    LatestBlessedModelStrategy,
)

resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],   # dari Evaluator tadi
).with_id("latest_blessed_model_resolver")

resolver_result = context.run(resolver)
resolver_result

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 45
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Resolver digunakan untuk memilih model terbaik sebelumnya (jika ada) berdasarkan blessing dari evaluator. Strategi yang digunakan adalah LatestBlessedModelStrategy. Jika tidak ada model sebelumnya, resolver mengembalikan None dan mengizinkan Trainer memasukkan model baru.

## 7) Pusher

In [12]:
from tfx.components.pusher.component import Pusher
from tfx.proto import pusher_pb2

pusher_spec = pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory=SERVING_MODEL_DIR
    )
)

pusher = Pusher(
    model=trainer.outputs["model"],
    model_blessing=evaluator.outputs["blessing"],
    push_destination=pusher_spec,
)

pusher_result = context.run(pusher)
pusher_result

ExecutionResult(
    component_id: Pusher
    execution_id: 46
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Pusher bertugas mendorong model yang sudah 'blessed' ke folder serving_model_dir. Model kemudian siap digunakan untuk TensorFlow Serving, FastAPI, Docker, atau platform lain.